## Deriving the Demographic Profile of United States Stakes 

Deriving the demographic profile of Stakes of the Church of Jesus Christ of Latter-day Saints involves mixing and matching several different data sources. 

1) Download the Census Bureau shapefiles for subcounty divisions (https://www.census.gov/cgi-bin/geo/shapefiles/index.php). These provide the next level of organization below County, and are sufficiently fine-grained for the East Coast context that I am in. For very geographically small stakes, such as those in Utah, smaller subdivisions such as Census blocks may be needed. For very small stakes there may not be demographic data available at that level. Whatever subdivision is used, there have to be enough to fit a lot of them into one stake.  

2) Go to Matt Martinich's Google Earth site (https://www.cumorah.com/index.php...) and download the KMZ file for wards (this is actually the one with the stake boundaries, not just points). Drop this into QGIS, then save it as a shape file. 

3) Create centroids for the subcounty division. This creates a point in the spatial center of the subcounty division. 

4) Perform a geospatial join for the centroids and the stake boundaries, then export the attribute table to a csv. This will provide a spreadsheet with a row for each county subdivision, as well as the stake in which its centroid is located. 

5) Download subcounty-specific variables from American Fact Finder. Choose which variables you are interested in. 

6) The code below is an example of how to create a weighted percentage, so if the subcounty division has half the population in the Stake, their income is worth half of the average, etc. This should provide a reasonably accurate demographic estimate for the stake. 

#### Load the necessary packages and set the working directory. 

In [3]:
import pandas as pd
import matplotlib 
import os
os.getcwd()
os.chdir("/Users/stephencranney/Desktop")

#### Load the data

In [4]:
Stakes = pd.read_csv("MarylandStakes.csv") #This is the merged stake file described above. 
SpanishLanguage= pd.read_csv("ACS_17_5YR_S1601_with_ann.csv", header=1) # This is the Census Bureau data for each county subdivision in that state. 

#### Merge the data 

In [11]:
Stakes['Id2']= Stakes['GEOID']
mergedDF=pd.merge(SpanishLanguage, Stakes, how='outer', on='Id2', indicator= 'exists')
#list(mergedDF.columns.values)

#### Calculate the weighted mean

In [21]:
mergedDF['SumPop'] = mergedDF['Total; Estimate; Population 5 years and over'].groupby(mergedDF['Name']).transform('sum')
mergedDF['prop']= mergedDF['Total; Estimate; Population 5 years and over'] / mergedDF['SumPop']

mergedDF['Spanish'] = pd.to_numeric(mergedDF['Percent; Estimate; SPEAK A LANGUAGE OTHER THAN ENGLISH - Spanish']) #, errors='coerce')
mergedDF['weighted_Spanish'] = mergedDF['Spanish'] * mergedDF['prop']
mergedDF['Stake_Spanish'] = mergedDF['weighted_Spanish'].groupby(mergedDF['Name']).transform('sum')


In [22]:
#Take the first 
Stakes2= mergedDF.groupby('Name').nth(1).reset_index()
Stakes3= Stakes2[['Stake_Spanish', 'Name']]
Stakes3



,Stake_Spanish,Name
0,5.878698,Annapolis Maryland Stake
1,3.607282,Baltimore Maryland Stake
2,0.577845,Clarksburg West Virginia Stake
3,4.599812,Columbia Maryland Stake
4,3.786565,Dover Delaware Stake
5,5.295459,Frederick Maryland Stake
6,2.833067,Martinsburg West Virginia Stake
7,18.309980,Seneca Maryland Stake
8,20.575152,Silver Spring Maryland Stake
9,4.384488,Suitland Maryland Stake


,Name,Id,Id2,Geography,Total; Estimate; Population 5 years and over,Total; Margin of Error; Population 5 years and over,Percent; Estimate; Population 5 years and over,Percent; Margin of Error; Population 5 years and over,"Percent of specified language speakers - Speak English only or speak English ""very well""; Estimate; Population 5 years and over","Percent of specified language speakers - Speak English only or speak English ""very well""; Margin of Error; Population 5 years and over",...,ALAND,AWATER,INTPTLAT,INTPTLON,exists,SumPop,prop,Spanish,weighted_Spanish,Stake_Spanish
0,Annapolis Maryland Stake,0600000US2400390100,2400390100,"District 2, Anne Arundel County, Maryland",72160,1612,(X),(X),68355,1590,...,61984747,852968,39.140405,-76.640375,both,521409,0.138394,4.6,0.636613,4.351993
1,Baltimore Maryland Stake,0600000US2400590564,2400590564,"District 7, Baltimore County, Maryland",8367,420,(X),(X),8291,424,...,147058102,423786,39.650693,-76.643324,both,1317379,0.006351,1.4,0.008892,2.617005
2,Clarksburg West Virginia Stake,0600000US2402390136,2402390136,"District 2, Friendsville; Garrett County; Mary...",1699,237,(X),(X),1699,237,...,132291945,2472013,39.671679,-79.398165,both,25552,0.066492,0.5,0.033246,0.316566
3,Columbia Maryland Stake,0600000US2400590104,2400590104,"District 2, Baltimore County, Maryland",98153,1810,(X),(X),94257,1934,...,121766489,3844778,39.364838,-76.816256,both,595362,0.164863,1.9,0.313239,3.324668
4,Dover Delaware Stake,0600000US2401190112,2401190112,"District 2, Greensboro; Caroline County; Maryland",5015,362,(X),(X),4898,353,...,131351346,587733,38.984729,-75.803353,both,307257,0.016322,2.7,0.044069,2.738908
5,Frederick Maryland Stake,0600000US2401390116,2401390116,"District 2, Uniontown; Carroll County; Maryland",3586,266,(X),(X),3574,265,...,84208432,303527,39.613458,-77.098739,both,351581,0.010200,0.5,0.005100,3.778336
6,Martinsburg West Virginia Stake,0600000US2400190096,2400190096,"District 2, Oldtown; Allegany County; Maryland",1207,232,(X),(X),1207,232,...,156974532,5055373,39.568845,-78.544429,both,212387,0.005683,3.4,0.019322,2.136532
7,Seneca Maryland Stake,0600000US2403190152,2403190152,"District 2, Montgomery County, Maryland",52347,1551,(X),(X),45797,1506,...,94929324,2333242,39.232971,-77.280281,both,296697,0.176433,8.8,1.552606,13.131258
8,Silver Spring Maryland Stake,0600000US2403390064,2403390064,"District 1, Vansville; Prince George's County;...",26919,1100,(X),(X),22192,975,...,44960983,270712,39.045689,-76.902979,both,556794,0.048346,15.5,0.749370,14.826974
9,Suitland Maryland Stake,0600000US2400990016,2400990016,"District 1, Solomons Island; Calvert County; M...",26110,884,(X),(X),25774,886,...,132879019,144184017,38.401554,-76.440952,both,807107,0.032350,1.6,0.051760,3.124000
